## 0.&nbsp;Instalación Spark

In [1]:
# Instalar SDK Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Descargar Spark 3.2.3
!wget -q https://archive.apache.org/dist/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz

# Descomprimir el archivo descargado de Spark
!tar xf spark-3.2.3-bin-hadoop3.2.tgz

# Establecer las variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"

# Instalar la librería findspark
!pip install -q findspark

# Instalar pyspark
!pip install -q pyspark==3.2.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.7 MB/s eta 0:00:00


## 1.&nbsp;Spark Session

In [3]:
# Find Spark
import findspark
findspark.init()

# Spark Session
from pyspark.sql import SparkSession

# SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## 2.&nbsp;Ejercicios

In [4]:
"""
# Ejercicio 1: Cree un RDD importes a partir de los datos adjuntos a esta
lección como recurso. Emplee acumuladores para obtener el total de
ventas realizadas y el importe total de las ventas.
"""
importes_rdd = sc.textFile('./rdd.txt')
importes_rdd = importes_rdd.map(lambda x: int(x))
importes_rdd.take(10)

[527, 386, 701, 240, 941, 27, 396, 56, 456, 148]

In [5]:
ventas_realizadas = sc.accumulator(0)
importes_rdd.foreach(lambda x: ventas_realizadas.add(1))
print(ventas_realizadas.value)

10000


In [6]:
importe_total = sc.accumulator(0)
importes_rdd.foreach(lambda x: importe_total.add(x))
print(importe_total.value)

5042335


In [7]:
"""
Ejercicio 2: Si se conoce que a cada venta hay que restarle un importe fijo
igual a 10 pesos por temas de impuestos.
a. ¿Cómo restaría este impuesto de cada venta utilizando una variable
broadcast para acelerar el proceso?
b. Cree un RDD llamado ventas_sin_impuestos a partir de la propuesta del inciso
a que contenga las ventas sin impuestos.
"""
impuesto = 10
impuesto_br = sc.broadcast(impuesto)
ventas_sin_impuestos_rdd = importes_rdd.map(lambda x: x - impuesto_br.value)
ventas_sin_impuestos_rdd.take(5)

[517, 376, 691, 230, 931]

In [8]:
"""
c. Destruya la variable broadcast creada luego de emplearla para crear
el RDD del inciso b.
"""
impuesto_br.destroy()

In [9]:
"""
Ejercicio 3 : Persista el RDD ventas_sin_impuestos en los siguientes
niveles de persistencia.
"""
from pyspark.storagelevel import StorageLevel

In [10]:
# a. Memoria.
ventas_sin_impuestos_rdd.persist(StorageLevel.MEMORY_ONLY)

PythonRDD[6] at RDD at PythonRDD.scala:53

In [11]:
ventas_sin_impuestos_rdd.unpersist()

PythonRDD[6] at RDD at PythonRDD.scala:53

In [12]:
# b. Disco solamente
ventas_sin_impuestos_rdd.persist(StorageLevel.DISK_ONLY)

PythonRDD[6] at RDD at PythonRDD.scala:53

In [13]:
ventas_sin_impuestos_rdd.unpersist()

PythonRDD[6] at RDD at PythonRDD.scala:53

In [14]:
# c. Memoria y disco.
ventas_sin_impuestos_rdd.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[6] at RDD at PythonRDD.scala:53